# VAE Training

## imports

In [6]:
import os, time
from images.Metrics import calculate_metrics
import numpy as np

from model.VariationalAutoencoder import VariationalAutoencoder

from keras.preprocessing.image import ImageDataGenerator, load_img, save_img, img_to_array, array_to_img

# environment variables

In [7]:
MODEL_FOLDER = os.environ.get('HOME') + '/model3/weights'
FILE_MODEL = 'weights.h5'
DATA_FOLDER = '/Volumes/My Passport/PFM/output/training_png'
RUN_FOLDER =  '/Volumes/My Passport/PFM/run/'

# VAE

## model variables

In [25]:
vae = VariationalAutoencoder(image_folder=DATA_FOLDER, run_folder=RUN_FOLDER, train_mode= False, z_dim=200, use_dropout=True, use_batch_norm=True)
vae.model.load_weights(os.path.join(MODEL_FOLDER, 'weights.h5'))  

In [29]:
from PIL import Image
from PIL.ImageShow import show

#image_path = '/Volumes/My Passport/PFM/output/training_png/Week9_39301_G11_4_Cells_34.png'
image_path = '/Volumes/My Passport/PFM/output/training_png/Week7_34641_E11_1_Cells_106.png'
#image_path = '/Volumes/My Passport/PFM/output/training_png/Week5_28921_B02_3_Cells_39.png'
#image_path = '/Volumes/My Passport/PFM/output/training_png/Week7_34641_B02_1_Cells_28.png'

img = Image.open(image_path)

#print("max: " + str(np.amax(array_img)))

#print(array_img)

#img = Image.open(os.sep.join([self.images_folder, 'training', i]))
array_img = img_to_array(img)  # keras
array_img = np.expand_dims(array_img, axis=0)
z_result = vae.encoder.predict(array_img)
reco = vae.decoder.predict(z_result).squeeze()
outimg = array_to_img(reco, scale=True)


img.show()
outimg.show()

# Metrics 

In [ ]:
calculate_metrics(vae = vae, path = DATA_FOLDER)